##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# Memuat Gambar

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/images"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Lihat di TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/images.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Jalankan di Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/images.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Lihat source di GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/images.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Unduh notebook</a>
  </td>
</table>

Tutorial ini menunjukkan sebuah contoh sederhana bagaimana cara memuat dataset gambar menggunakan `tf.data`.

Dataset yang digunakan dalam contoh ini terdistribusi sebagai kumpulan direktori dari gambar, dengan satu kelas gambar setiap direktori.

## Pengaturan

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
try:
  # %tensorflow_version hanya tersedia di Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
tf.__version__

### Memperoleh gambar

Sebelum Anda memulai proses training, Anda membutuhkan sebuah set gambar untuk melatih neural network tentang kelas-kelas baru yang ingin Anda kenali. Anda dapat menggunakan arsip dari foto bunga berlisensi dari Google.

Catatan: semua gambar terlisensi CC-BY, para pembuat terdaftar di file `LICENSE.txt`.

In [ ]:
import pathlib
data_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)
data_dir = pathlib.Path(data_dir)

Setelah mengunduh (218MB), Anda sekarang memiliki salinan gambar-gambar bunga yang tersedia.

Direktori tersebut terdiri atas 5 subdirektori, satu setiap kelas:

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count

In [ ]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
CLASS_NAMES

Setiap direktori terdiri atas gambar-gambar dari tipe bunga. Ini adalah gambar beberapa mawar:

In [ ]:
roses = list(data_dir.glob('roses/*'))

for image_path in roses[:3]:
    display.display(Image.open(str(image_path)))

## Memuat menggunakan `keras.preprocessing`

Cara sederhana untuk memuat gambar adalah dengan menggunakan `tf.keras.preprocessing`.

In [ ]:
# 1./255 dilakukan untuk mengubah dari uint8 menjadi float32 dalam rentang [0,1].
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

Mendefinisikan beberapa parameter untuk loader:

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

In [ ]:
train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES))

Pengamatan sebuah batch:

In [ ]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

In [ ]:
image_batch, label_batch = next(train_data_gen)
show_batch(image_batch, label_batch)

## Memuat menggunakan `tf.data`

The above `keras.preprocessing` method is convienient, but has two downsides: 
Metode `keras.preprocessing` di atas sangat mudah digunakan, tetapi memiliki beberapa kekurangan:

1. Metode ini lambat. Lihat bagian performa di bawah.
1. Kurang kontrol yang lebih detail.
1. Metode tersebut tidak terintegrasi secara sempurna dengan keseluruhan TensorFlow.

Untuk memuat file sebagai sebuah `tf.data.Dataset` pertama-tama buat sebuah dataset dari path file:

In [ ]:
list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))

In [ ]:
for f in list_ds.take(5):
  print(f.numpy())

Tulis sebuah fungsi pendek tensorflow yang mengubah sebuah path file menjadi pasangan (image_data, label):

In [ ]:
def get_label(file_path):
  # ubah path menjadi list komponen path
  parts = tf.strings.split(file_path, os.path.sep)
  # dua dari akhir adalah direktori kelas
  return parts[-2] == CLASS_NAMES

In [ ]:
def decode_img(img):
  # ubah string terkompresi menjadi tensor uint8 3D
  img = tf.image.decode_jpeg(img, channels=3)
  # Gunakan `convert_image_dtype` untuk mengkonversi menjadi float dengan range [0,1].
  img = tf.image.convert_image_dtype(img, tf.float32)
  # Atur ukuran image menjadi ukuran yang diinginkan
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [ ]:
def process_path(file_path):
  label = get_label(file_path)
  # Muat raw data dari file sebagai sebuah string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

Gunakan `Dataset.map` untuk membuat sebuah dataset dari pasangan `image, label`

In [ ]:
# Set `num_parallel_calls` sehingga beberapa gambar dimuat/diproses secara paralel
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
for image, label in labeled_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

### Metode dasar untuk proses training

To train a model with this dataset you will want the data:
Untuk melatih sebuah model dengan dataset ini, Anda ingin data:

* Ter-*shuffle* dengan baik.
* Dalam batch.
* Batch-batch tersedia secepatnya.

Fitur-fitur ini dapat dengan mudah ditambahkan menggunakan api `tf.data`

In [ ]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  # Dataset ini berukuran kecil, muat dataset sekali, dan simpan dalam memori.
  # Gunakan `.cache(filename)` untuk melakukan proses *cache preprocessing* dataset yang tidak 
  # muat di memori.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # ulangi selamanya
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` memungkinan dataset mengambil batch pada background ketika model
  # ditraining
    
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

In [ ]:
train_ds = prepare_for_training(labeled_ds)

image_batch, label_batch = next(iter(train_ds))

In [ ]:
show_batch(image_batch.numpy(), label_batch.numpy())

## Performa

Catatan: Bagian ini hanya menunjukkan beberapa trik mudah yang mungkin dapat membantu performa. Untuk petunjuk yang lebih dalam lihat [Inputasi Performa Pipeline](../../guide/performance/datasets).

Untuk pengamatan, pertama-tama buat sebuah fungsi untuk mengecek performa dari dataset kita:

In [ ]:
import time
default_timeit_steps = 1000

def timeit(ds, steps=default_timeit_steps):
  start = time.time()
  it = iter(ds)
  for i in range(steps):
    batch = next(it)
    if i%10 == 0:
      print('.',end='')
  print()
  end = time.time()

  duration = end-start
  print("{} batches: {} s".format(steps, duration))
  print("{:0.5f} Images/s".format(BATCH_SIZE*steps/duration))

Mari kita bandingkan kecepatan dari dua generator data:

In [ ]:
# `keras.preprocessing`
timeit(train_data_gen)

In [ ]:
# `tf.data`
timeit(train_ds)

Sebagian besar perolehan performa datang dari penggunaan `.cache`.

In [ ]:
uncached_ds = prepare_for_training(labeled_ds, cache=False)
timeit(uncached_ds)

Apabila dataset tidak muat dalam memori gunakan sebuah file cache untuk mendapatkan beberapa manfaat:

In [ ]:
filecache_ds = prepare_for_training(labeled_ds, cache="./flowers.tfcache")
timeit(filecache_ds)